# * VINSIGHT : Special Metric

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Revenue Corporate (C, P)

In [2]:
''' Execute transaction '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> Data as of {curr_datetime}')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B0R000100' --Total Revenue
        , 'DB0R000100' --Total Revenue : DTAC
        , 'TB0R000100' --Total Revenue : TRUE
        , 'B0R000100GEO' --Total Revenue (Geo)
        , 'DB0R000100GEO' --Total Revenue (Geo) : DTAC
        , 'TB0R000100GEO' --Total Revenue (Geo) : TRUE

        , 'B0R000101' --Mobile Revenue
        , 'DB0R000101' --Mobile Revenue : DTAC
        , 'TB0R000101' --Mobile Revenue : TMH
        , 'B0R000101GEO' --Mobile Revenue (Geo)
        , 'DB0R000101GEO' --Mobile Revenue (Geo) : DTAC
        , 'TB0R000101GEO' --Mobile Revenue (Geo) : TMH
        
        , 'B2R000100' --Postpaid Revenue
        , 'B2R010100' --Postpaid Revenue B2C
        , 'B2R020100' --Postpaid Revenue B2B
        , 'DB2R000100' --Postpaid Revenue : DTAC
        , 'DB2R010100' --Postpaid Revenue B2C : DTAC
        , 'DB2R020100' --Postpaid Revenue B2B : DTAC
        , 'TB2R000100' --Postpaid Revenue : TMH
        , 'TB2R010100' --Postpaid Revenue B2C : TMH
        , 'TB2R020100' --Postpaid Revenue B2B : TMH
        
        , 'B1R000100' --Prepaid Revenue
        , 'DB1R000100' --Prepaid Revenue : DTAC
        , 'TB1R000100' --Prepaid Revenue : TMH
        
        , 'TB3R000100' --TOL Revenue
        
        , 'TB4R000100' --TVS Revenue
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...

   -> Data as of 2025-05-30, 11:37:04
   -> v_start_date: 20250101


In [3]:
''' Automate Month Period '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = str(chk_src_df['TM_KEY_MTH'].drop_duplicates().astype(int).shift().max())

In [4]:
''' Monthly Summary '''

monthly_df = chk_src_df.copy()
monthly_df = monthly_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
monthly_df = monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
monthly_df['DIFF (C-P)'] = monthly_df['C'] - monthly_df['P']

monthly_df_display = monthly_df.copy()
mod_col_list = monthly_df_display.iloc[:, 5:].columns.tolist()
for col in mod_col_list:
    monthly_df_display[col] = monthly_df_display[col].apply(lambda x: format(x, ',.0f'))

# monthly_df_display

In [5]:
# ''' Sample Data '''

# sample_mth_df = monthly_df_display.loc[monthly_df_display['TM_KEY_MTH']==202501]
# sample_mth_df

### Peparing Check Sum Data

In [6]:
''' B0R000100 : Total Revenue '''

v_metric_cd = 'B0R000100'
v_metric_name = 'Total Revenue (sum)'
v_metric_list = [
    'DB1R000100' # Prepaid Revenue : DTAC
    , 'TB1R000100' # Prepaid Revenue : TMH
    , 'DB2R000100' # Postpaid Revenue : DTAC
    , 'TB2R000100' # Postpaid Revenue : TMH
    , 'TB3R000100' # TOL Revenue
    , 'TB4R000100' # TVS Revenue
    ]

chk_B0R000100_df = monthly_df.copy()
chk_B0R000100_df = chk_B0R000100_df.loc[chk_B0R000100_df['METRIC_CD'].isin(v_metric_list)]
chk_B0R000100_df['METRIC_CD'] = v_metric_cd
chk_B0R000100_df['METRIC_NAME'] = v_metric_name
chk_B0R000100_df = chk_B0R000100_df.groupby(['TM_KEY_MTH', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum'})
chk_B0R000100_df.rename(columns={'C':'C (Sum)', 'P':'P (Sum)'}, inplace=True)
chk_B0R000100_df = chk_B0R000100_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()

chk_B0R000100_df_display = chk_B0R000100_df.copy()
mod_col_list = chk_B0R000100_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    chk_B0R000100_df_display[col] = chk_B0R000100_df_display[col].apply(lambda x: format(x, ',.0f'))
chk_B0R000100_df_display

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,C (Sum),P (Sum)
0,202501,B0R000100,Total Revenue (sum),"12,770,106,773","12,636,945,503"
1,202502,B0R000100,Total Revenue (sum),"12,236,601,216","12,212,436,891"
2,202503,B0R000100,Total Revenue (sum),"12,516,306,281","12,587,616,315"
3,202504,B0R000100,Total Revenue (sum),"12,526,105,797","12,501,905,390"
4,202505,B0R000100,Total Revenue (sum),"11,168,468,778","11,141,939,416"


In [7]:
''' TB0R000100 : Total Revenue : TMH '''

v_metric_cd = 'TB0R000100'
v_metric_name = 'Total Revenue : TMH (sum)'
v_metric_list = [
    'TB1R000100' # Prepaid Revenue : TMH
    , 'TB2R000100' # Postpaid Revenue : TMH
    , 'TB3R000100' # TOL Revenue
    , 'TB4R000100' # TVS Revenue
    ]

chk_TB0R000100_df = monthly_df.copy()
chk_TB0R000100_df = chk_TB0R000100_df.loc[chk_TB0R000100_df['METRIC_CD'].isin(v_metric_list)]
chk_TB0R000100_df['METRIC_CD'] = v_metric_cd
chk_TB0R000100_df['METRIC_NAME'] = v_metric_name
chk_TB0R000100_df = chk_TB0R000100_df.groupby(['TM_KEY_MTH', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum'})
chk_TB0R000100_df.rename(columns={'C':'C (Sum)', 'P':'P (Sum)'}, inplace=True)
chk_TB0R000100_df = chk_TB0R000100_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()

chk_TB0R000100_df_display = chk_TB0R000100_df.copy()
mod_col_list = chk_TB0R000100_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    chk_TB0R000100_df_display[col] = chk_TB0R000100_df_display[col].apply(lambda x: format(x, ',.0f'))
chk_TB0R000100_df_display

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,C (Sum),P (Sum)
0,202501,TB0R000100,Total Revenue : TMH (sum),"8,190,378,845","8,057,003,936"
1,202502,TB0R000100,Total Revenue : TMH (sum),"7,860,591,688","7,836,255,273"
2,202503,TB0R000100,Total Revenue : TMH (sum),"8,005,985,591","8,077,214,145"
3,202504,TB0R000100,Total Revenue : TMH (sum),"8,067,988,282","8,043,775,917"
4,202505,TB0R000100,Total Revenue : TMH (sum),"7,139,952,285","7,113,417,087"


In [8]:
''' DB0R000100 : Total Revenue : DTAC '''

v_metric_cd = 'DB0R000100'
v_metric_name = 'Total Revenue : DTAC (sum)'
v_metric_list = [
    'DB1R000100' # Prepaid Revenue : DTAC
    , 'DB2R000100' # Postpaid Revenue : DTAC
    ]

chk_DB0R000100_df = monthly_df.copy()
chk_DB0R000100_df = chk_DB0R000100_df.loc[chk_DB0R000100_df['METRIC_CD'].isin(v_metric_list)]
chk_DB0R000100_df['METRIC_CD'] = v_metric_cd
chk_DB0R000100_df['METRIC_NAME'] = v_metric_name
chk_DB0R000100_df = chk_DB0R000100_df.groupby(['TM_KEY_MTH', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum'})
chk_DB0R000100_df.rename(columns={'C':'C (Sum)', 'P':'P (Sum)'}, inplace=True)
chk_DB0R000100_df = chk_DB0R000100_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()

chk_DB0R000100_df_display = chk_DB0R000100_df.copy()
mod_col_list = chk_DB0R000100_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    chk_DB0R000100_df_display[col] = chk_DB0R000100_df_display[col].apply(lambda x: format(x, ',.0f'))
chk_DB0R000100_df_display

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,C (Sum),P (Sum)
0,202501,DB0R000100,Total Revenue : DTAC (sum),"4,579,727,928","4,579,941,567"
1,202502,DB0R000100,Total Revenue : DTAC (sum),"4,376,009,529","4,376,181,618"
2,202503,DB0R000100,Total Revenue : DTAC (sum),"4,510,320,691","4,510,402,170"
3,202504,DB0R000100,Total Revenue : DTAC (sum),"4,458,117,515","4,458,129,472"
4,202505,DB0R000100,Total Revenue : DTAC (sum),"4,028,516,493","4,028,522,329"


In [9]:
''' B0R000101 : Mobile Revenue '''

v_metric_cd = 'B0R000101'
v_metric_name = 'Mobile Revenue (sum)'
v_metric_list = [
    'DB1R000100' # Prepaid Revenue : DTAC
    , 'TB1R000100' # Prepaid Revenue : TMH
    , 'DB2R000100' # Postpaid Revenue : DTAC
    , 'TB2R000100' # Postpaid Revenue : TMH
    ]

chk_B0R000101_df = monthly_df.copy()
chk_B0R000101_df = chk_B0R000101_df.loc[chk_B0R000101_df['METRIC_CD'].isin(v_metric_list)]
chk_B0R000101_df['METRIC_CD'] = v_metric_cd
chk_B0R000101_df['METRIC_NAME'] = v_metric_name
chk_B0R000101_df = chk_B0R000101_df.groupby(['TM_KEY_MTH', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum'})
chk_B0R000101_df.rename(columns={'C':'C (Sum)', 'P':'P (Sum)'}, inplace=True)
chk_B0R000101_df = chk_B0R000101_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()

chk_B0R000101_df_display = chk_B0R000101_df.copy()
mod_col_list = chk_B0R000101_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    chk_B0R000101_df_display[col] = chk_B0R000101_df_display[col].apply(lambda x: format(x, ',.0f'))
chk_B0R000101_df_display

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,C (Sum),P (Sum)
0,202501,B0R000101,Mobile Revenue (sum),"10,755,733,702","10,751,551,183"
1,202502,B0R000101,Mobile Revenue (sum),"10,319,699,474","10,314,001,696"
2,202503,B0R000101,Mobile Revenue (sum),"10,685,245,593","10,678,120,361"
3,202504,B0R000101,Mobile Revenue (sum),"10,592,109,386","10,585,358,700"
4,202505,B0R000101,Mobile Revenue (sum),"9,540,131,126","9,526,963,581"


In [10]:
''' TB0R000101 : Mobile Revenue : TMH '''

v_metric_cd = 'TB0R000101'
v_metric_name = 'Mobile Revenue : TMH (sum)'
v_metric_list = [
    'TB1R000100' # Prepaid Revenue : TMH
    , 'TB2R000100' # Postpaid Revenue : TMH
    ]

chk_TB0R000101_df = monthly_df.copy()
chk_TB0R000101_df = chk_TB0R000101_df.loc[chk_TB0R000101_df['METRIC_CD'].isin(v_metric_list)]
chk_TB0R000101_df['METRIC_CD'] = v_metric_cd
chk_TB0R000101_df['METRIC_NAME'] = v_metric_name
chk_TB0R000101_df = chk_TB0R000101_df.groupby(['TM_KEY_MTH', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum'})
chk_TB0R000101_df.rename(columns={'C':'C (Sum)', 'P':'P (Sum)'}, inplace=True)
chk_TB0R000101_df = chk_TB0R000101_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()

chk_TB0R000101_df_display = chk_TB0R000101_df.copy()
mod_col_list = chk_TB0R000101_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    chk_TB0R000101_df_display[col] = chk_TB0R000101_df_display[col].apply(lambda x: format(x, ',.0f'))
chk_TB0R000101_df_display

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,C (Sum),P (Sum)
0,202501,TB0R000101,Mobile Revenue : TMH (sum),"6,176,005,774","6,171,609,616"
1,202502,TB0R000101,Mobile Revenue : TMH (sum),"5,943,689,946","5,937,820,079"
2,202503,TB0R000101,Mobile Revenue : TMH (sum),"6,174,924,902","6,167,718,192"
3,202504,TB0R000101,Mobile Revenue : TMH (sum),"6,133,991,871","6,127,229,228"
4,202505,TB0R000101,Mobile Revenue : TMH (sum),"5,511,614,633","5,498,441,251"


In [11]:
''' DB0R000101 : Mobile Revenue : DTAC '''

v_metric_cd = 'DB0R000101'
v_metric_name = 'Mobile Revenue : DTAC (sum)'
v_metric_list = [
    'DB1R000100' # Prepaid Revenue : DTAC
    , 'DB2R000100' # Postpaid Revenue : DTAC
    ]

chk_DB0R000101_df = monthly_df.copy()
chk_DB0R000101_df = chk_DB0R000101_df.loc[chk_DB0R000101_df['METRIC_CD'].isin(v_metric_list)]
chk_DB0R000101_df['METRIC_CD'] = v_metric_cd
chk_DB0R000101_df['METRIC_NAME'] = v_metric_name
chk_DB0R000101_df = chk_DB0R000101_df.groupby(['TM_KEY_MTH', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum'})
chk_DB0R000101_df.rename(columns={'C':'C (Sum)', 'P':'P (Sum)'}, inplace=True)
chk_DB0R000101_df = chk_DB0R000101_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()

chk_DB0R000101_df_display = chk_DB0R000101_df.copy()
mod_col_list = chk_DB0R000101_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    chk_DB0R000101_df_display[col] = chk_DB0R000101_df_display[col].apply(lambda x: format(x, ',.0f'))
chk_DB0R000101_df_display

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,C (Sum),P (Sum)
0,202501,DB0R000101,Mobile Revenue : DTAC (sum),"4,579,727,928","4,579,941,567"
1,202502,DB0R000101,Mobile Revenue : DTAC (sum),"4,376,009,529","4,376,181,618"
2,202503,DB0R000101,Mobile Revenue : DTAC (sum),"4,510,320,691","4,510,402,170"
3,202504,DB0R000101,Mobile Revenue : DTAC (sum),"4,458,117,515","4,458,129,472"
4,202505,DB0R000101,Mobile Revenue : DTAC (sum),"4,028,516,493","4,028,522,329"


In [12]:
''' Concat All Check Sum Data '''

sum_rev_corp_df = pd.concat([chk_B0R000100_df, chk_TB0R000100_df, chk_DB0R000100_df, chk_B0R000101_df, chk_TB0R000101_df, chk_DB0R000101_df])
# sum_rev_corp_df

### Comparison Overview

In [13]:
''' Total Revenue & Mobile Revenue '''

v_metric_list = [
    'B0R000100' # Total Revenue
    , 'DB0R000100' # Total Revenue : DTAC
    , 'TB0R000100' # Total Revenue : TRUE
    , 'B0R000101' # Mobile Revenue
    , 'DB0R000101' # Mobile Revenue : DTAC
    , 'TB0R000101' # Mobile Revenue : TMH
    ]

vis_rev_corp_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C', 'P']].copy() #, 'DIFF (C-P)'
vis_rev_corp_df = vis_rev_corp_df.loc[vis_rev_corp_df['METRIC_CD'].isin(v_metric_list)]

merge_comparison_df = pd.merge(vis_rev_corp_df, sum_rev_corp_df[['TM_KEY_MTH', 'METRIC_CD', 'C (Sum)', 'P (Sum)']], how='left', on=['TM_KEY_MTH', 'METRIC_CD'])
merge_comparison_df['DIFF_C'] = merge_comparison_df['C'] - merge_comparison_df['C (Sum)']
merge_comparison_df['DIFF_P'] = merge_comparison_df['P'] - merge_comparison_df['P (Sum)']

comparison_df = merge_comparison_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index(drop=True)

comparison_df_display = comparison_df.copy()
mod_col_list = comparison_df_display.iloc[:, 5:].columns.tolist()
for col in mod_col_list:
    comparison_df_display[col] = comparison_df_display[col].apply(lambda x: format(x, ',.0f'))

comparison_df_display

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,C (Sum),P (Sum),DIFF_C,DIFF_P
0,202501,All Services,B0R000100,Total Revenue,2025-05-29 23:54:18,"15,530,009,361","12,636,945,503","12,770,106,773","12,636,945,503","2,759,902,588",0
1,202501,All Services,DB0R000100,Total Revenue : DTAC,2025-05-29 23:54:18,"7,339,630,516","4,579,941,567","4,579,727,928","4,579,941,567","2,759,902,588",0
2,202501,All Services,TB0R000100,Total Revenue : TRUE,2025-05-29 23:54:18,"8,190,378,845","8,057,003,936","8,190,378,845","8,057,003,936",0,0
3,202501,Mobile,B0R000101,Mobile Revenue,2025-05-29 23:54:18,"13,515,636,290","10,751,551,183","10,755,733,702","10,751,551,183","2,759,902,588",0
4,202501,Mobile,DB0R000101,Mobile Revenue : DTAC,2025-05-29 23:54:18,"7,339,630,516","4,579,941,567","4,579,727,928","4,579,941,567","2,759,902,588",0
5,202501,Mobile,TB0R000101,Mobile Revenue : TMH,2025-05-29 23:54:18,"6,176,005,774","6,171,609,616","6,176,005,774","6,171,609,616",0,0
6,202502,All Services,B0R000100,Total Revenue,2025-05-29 23:54:18,"14,637,318,577","12,212,436,891","12,236,601,216","12,212,436,891","2,400,717,361",0
7,202502,All Services,DB0R000100,Total Revenue : DTAC,2025-05-29 23:54:18,"6,776,726,890","4,376,181,618","4,376,009,529","4,376,181,618","2,400,717,361",0
8,202502,All Services,TB0R000100,Total Revenue : TRUE,2025-05-29 23:54:18,"7,860,591,688","7,836,255,273","7,860,591,688","7,836,255,273",0,0
9,202502,Mobile,B0R000101,Mobile Revenue,2025-05-29 23:54:18,"12,720,416,836","10,314,001,696","10,319,699,474","10,314,001,696","2,400,717,361",0


### Comparison by Month

In [14]:
y202501_comparison_df = comparison_df_display.loc[comparison_df_display['TM_KEY_MTH']==202501]
y202501_comparison_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,C (Sum),P (Sum),DIFF_C,DIFF_P
0,202501,All Services,B0R000100,Total Revenue,2025-05-29 23:54:18,"15,530,009,361","12,636,945,503","12,770,106,773","12,636,945,503","2,759,902,588",0
1,202501,All Services,DB0R000100,Total Revenue : DTAC,2025-05-29 23:54:18,"7,339,630,516","4,579,941,567","4,579,727,928","4,579,941,567","2,759,902,588",0
2,202501,All Services,TB0R000100,Total Revenue : TRUE,2025-05-29 23:54:18,"8,190,378,845","8,057,003,936","8,190,378,845","8,057,003,936",0,0
3,202501,Mobile,B0R000101,Mobile Revenue,2025-05-29 23:54:18,"13,515,636,290","10,751,551,183","10,755,733,702","10,751,551,183","2,759,902,588",0
4,202501,Mobile,DB0R000101,Mobile Revenue : DTAC,2025-05-29 23:54:18,"7,339,630,516","4,579,941,567","4,579,727,928","4,579,941,567","2,759,902,588",0
5,202501,Mobile,TB0R000101,Mobile Revenue : TMH,2025-05-29 23:54:18,"6,176,005,774","6,171,609,616","6,176,005,774","6,171,609,616",0,0


In [15]:
y202502_comparison_df = comparison_df_display.loc[comparison_df_display['TM_KEY_MTH']==202502]
y202502_comparison_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,C (Sum),P (Sum),DIFF_C,DIFF_P
6,202502,All Services,B0R000100,Total Revenue,2025-05-29 23:54:18,"14,637,318,577","12,212,436,891","12,236,601,216","12,212,436,891","2,400,717,361",0
7,202502,All Services,DB0R000100,Total Revenue : DTAC,2025-05-29 23:54:18,"6,776,726,890","4,376,181,618","4,376,009,529","4,376,181,618","2,400,717,361",0
8,202502,All Services,TB0R000100,Total Revenue : TRUE,2025-05-29 23:54:18,"7,860,591,688","7,836,255,273","7,860,591,688","7,836,255,273",0,0
9,202502,Mobile,B0R000101,Mobile Revenue,2025-05-29 23:54:18,"12,720,416,836","10,314,001,696","10,319,699,474","10,314,001,696","2,400,717,361",0
10,202502,Mobile,DB0R000101,Mobile Revenue : DTAC,2025-05-29 23:54:18,"6,776,726,890","4,376,181,618","4,376,009,529","4,376,181,618","2,400,717,361",0
11,202502,Mobile,TB0R000101,Mobile Revenue : TMH,2025-05-29 23:54:18,"5,943,689,946","5,937,820,079","5,943,689,946","5,937,820,079",0,0


In [16]:
y202503_comparison_df = comparison_df_display.loc[comparison_df_display['TM_KEY_MTH']==202503]
y202503_comparison_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,C (Sum),P (Sum),DIFF_C,DIFF_P
12,202503,All Services,B0R000100,Total Revenue,2025-05-29 23:54:18,"15,251,112,086","12,587,616,315","12,516,306,281","12,587,616,315","2,734,805,804",0
13,202503,All Services,DB0R000100,Total Revenue : DTAC,2025-05-29 23:54:18,"7,245,126,495","4,510,402,170","4,510,320,691","4,510,402,170","2,734,805,804",0
14,202503,All Services,TB0R000100,Total Revenue : TRUE,2025-05-29 23:54:18,"8,005,985,591","8,077,214,145","8,005,985,591","8,077,214,145",0,0
15,202503,Mobile,B0R000101,Mobile Revenue,2025-05-29 23:54:18,"13,420,051,397","10,678,120,361","10,685,245,593","10,678,120,361","2,734,805,804",0
16,202503,Mobile,DB0R000101,Mobile Revenue : DTAC,2025-05-29 23:54:18,"7,245,126,495","4,510,402,170","4,510,320,691","4,510,402,170","2,734,805,804",0
17,202503,Mobile,TB0R000101,Mobile Revenue : TMH,2025-05-29 23:54:18,"6,174,924,902","6,167,718,192","6,174,924,902","6,167,718,192",0,0


In [17]:
y202504_comparison_df = comparison_df_display.loc[comparison_df_display['TM_KEY_MTH']==202504]
y202504_comparison_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,C (Sum),P (Sum),DIFF_C,DIFF_P
18,202504,All Services,B0R000100,Total Revenue,2025-05-29 23:54:18,"12,526,105,797","12,501,905,390","12,526,105,797","12,501,905,390",0,0
19,202504,All Services,DB0R000100,Total Revenue : DTAC,2025-05-29 23:54:18,"4,458,117,515","4,458,129,472","4,458,117,515","4,458,129,472",0,0
20,202504,All Services,TB0R000100,Total Revenue : TRUE,2025-05-29 23:54:18,"8,067,988,282","8,043,775,917","8,067,988,282","8,043,775,917",0,0
21,202504,Mobile,B0R000101,Mobile Revenue,2025-05-29 23:54:18,"10,592,109,386","10,585,358,700","10,592,109,386","10,585,358,700",0,0
22,202504,Mobile,DB0R000101,Mobile Revenue : DTAC,2025-05-29 23:54:18,"4,458,117,515","4,458,129,472","4,458,117,515","4,458,129,472",-0,0
23,202504,Mobile,TB0R000101,Mobile Revenue : TMH,2025-05-29 23:54:18,"6,133,991,871","6,127,229,228","6,133,991,871","6,127,229,228",0,0


In [18]:
y202505_comparison_df = comparison_df_display.loc[comparison_df_display['TM_KEY_MTH']==202505]
y202505_comparison_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,C (Sum),P (Sum),DIFF_C,DIFF_P
24,202505,All Services,B0R000100,Total Revenue,2025-05-29 23:54:18,"11,168,468,778","11,141,939,416","11,168,468,778","11,141,939,416",0,0
25,202505,All Services,DB0R000100,Total Revenue : DTAC,2025-05-29 23:54:18,"4,028,516,493","4,028,522,329","4,028,516,493","4,028,522,329",0,0
26,202505,All Services,TB0R000100,Total Revenue : TRUE,2025-05-29 23:54:18,"7,139,952,285","7,113,417,087","7,139,952,285","7,113,417,087",0,0
27,202505,Mobile,B0R000101,Mobile Revenue,2025-05-29 23:54:18,"9,540,131,126","9,526,963,581","9,540,131,126","9,526,963,581",0,0
28,202505,Mobile,DB0R000101,Mobile Revenue : DTAC,2025-05-29 23:54:18,"4,028,516,493","4,028,522,329","4,028,516,493","4,028,522,329",0,0
29,202505,Mobile,TB0R000101,Mobile Revenue : TMH,2025-05-29 23:54:18,"5,511,614,633","5,498,441,251","5,511,614,633","5,498,441,251",0,0


In [19]:
y202506_comparison_df = comparison_df_display.loc[comparison_df_display['TM_KEY_MTH']==202506]
y202506_comparison_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,C (Sum),P (Sum),DIFF_C,DIFF_P


### Pending...

In [20]:
# ''' B0R000100 : Total Revenue '''

# v_total_metric_list = [
#     'DB1R000100' # Prepaid Revenue : DTAC
#     , 'TB1R000100' # Prepaid Revenue : TMH
#     , 'DB2R000100' # Postpaid Revenue : DTAC
#     , 'TB2R000100' # Postpaid Revenue : TMH
#     , 'TB3R000100' # TOL Revenue
#     , 'TB4R000100' # TVS Revenue
#     ]

# B0R000100_day_df = chk_src_df.copy()
# B0R000100_day_df = B0R000100_day_df.loc[B0R000100_day_df['METRIC_CD'].isin(v_total_metric_list)]

# # B0R000100_day_df = chk_src_df.loc[chk_src_df['TM_KEY_MTH']==202501].copy()
# B0R000100_day_df['METRIC_TMP'] = B0R000100_day_df['METRIC_NAME'].replace(r' : DTAC| : TRUE| : TMH', '', regex=True)
# B0R000100_day_df['ALL'] = np.where(B0R000100_day_df['COMP_CD']=='ALL', B0R000100_day_df['C'], 0)
# B0R000100_day_df['TRUE'] = np.where(B0R000100_day_df['COMP_CD']=='TRUE', B0R000100_day_df['C'], 0)
# B0R000100_day_df['DTAC'] = np.where(B0R000100_day_df['COMP_CD']=='DTAC', B0R000100_day_df['C'], 0)

# B0R000100_day_df = B0R000100_day_df.groupby(['TM_KEY_MTH', 'METRIC_TMP']).agg({'PPN_TM':'max', 'ALL':'sum', 'TRUE':'sum', 'DTAC':'sum'})
# B0R000100_day_df = B0R000100_day_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'METRIC_TMP']).reset_index()

# B0R000100_day_df['SUM (T+D)'] = B0R000100_day_df['TRUE'] + B0R000100_day_df['DTAC']
# B0R000100_day_df['DIFF (SUM-ALL)'] = B0R000100_day_df['SUM (T+D)'] - B0R000100_day_df['ALL']

# mod_col_list = B0R000100_day_df.iloc[:, 3:8].columns.tolist()
# for col in mod_col_list:
#     B0R000100_day_df[col] = B0R000100_day_df[col].apply(lambda x: format(x, ',.0f'))

# B0R000100_day_df